In [1]:
import pandas as pd

pd.options.display.max_rows = 100000
pd.set_option('display.max_colwidth', -1)

C:\Users\bdolicki\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [7]:
PATH="../analysing_features/udpos/results/predictions/"

In [9]:
with open(f"{PATH}dev-en.tsv",encoding='utf-8') as f:
    lines = f.read().split("\n")

all_sentences = []
all_tags = []
sentence = ""
sentene_tags = ""
i = 0
for line in lines:
#     if a sentence ended
    if line=="":
        i += 1
        all_sentences.append(sentence.strip())
        all_tags.append(sentene_tags.strip())
        sentence =""
        sentene_tags = ""
    else:
        word = line.split("\t")[0]
        tag = line.split("\t")[1]
        
        sentence += " "+word
        sentene_tags += " "+tag

In [11]:
with open(f"{PATH}dev_en_predictions.txt") as f:
    lines = f.read().split("\n")

all_pred_tags = []
pred_tags = ""
for line in lines:
#     if a sentence ended
    if line=="":
        all_pred_tags.append(pred_tags.strip())
        pred_tags = ""
    else:
        pred_tags += " "+line

In [12]:
ud_pos = pd.DataFrame(data={"sentences":all_sentences,"ground_truth":all_tags,"predictions":all_pred_tags})

In [13]:
unique_tags = "ADJ ADP ADV AUX CCONJ DET INTJ NOUN NUM PART PRON PROPN PUNCT SCONJ SYM VERB X".split()

In [14]:
import numpy as np
confusion_matrix = np.zeros((len(unique_tags),len(unique_tags)))
# rows represent predicted values and columns represent actuals

In [15]:
total_tags = 0
incorrect_tags = 0
mistakes_df = {"predicted":[],"actual":[],"token":[],"sentence":[],"actual_tokens":[]}
for i in range(len(all_tags)):
    actual_sent = all_tags[i].split()
    pred_sent = all_pred_tags[i].split()
    assert len(actual_sent)==len(pred_sent),"Number of actual and predicted tags for the same sentence should be equal"
    for j in range(len(actual_sent)):
        if not actual_sent[j]==pred_sent[j]:
            mistakes_df["predicted"].append(pred_sent[j])
            mistakes_df["actual"].append(actual_sent[j])
            mistakes_df["token"].append(all_sentences[i].split()[j])
            mistakes_df["sentence"].append(all_sentences[i])
            mistakes_df["actual_tokens"].append(" ".join(actual_sent))
            
            confusion_matrix[unique_tags.index(pred_sent[j])][unique_tags.index(actual_sent[j])] += 1
            incorrect_tags += 1
        total_tags += 1
#     if incorrect_tags==5:
#         print("break")
#         break

In [16]:
mistakes_df = pd.DataFrame.from_dict(mistakes_df)

In [19]:
conf_df = pd.DataFrame(data=confusion_matrix,index=unique_tags,columns=unique_tags)

#### How many times each tag is misclassified

In [20]:
conf_count = pd.DataFrame()
conf_count["count"] = conf_df.sum().sort_values(ascending=False); 
conf_count["percent"] = conf_count/conf_count.sum()
conf_count[:10]

,count,percent
NOUN,10375.0,0.244763
VERB,5700.0,0.134472
ADP,4961.0,0.117038
DET,4623.0,0.109064
PRON,3750.0,0.088468
ADJ,3580.0,0.084458
AUX,2562.0,0.060442
ADV,2011.0,0.047443
PART,1391.0,0.032816
PROPN,1225.0,0.028900


In [21]:
sparse_conf_df = conf_df.reset_index().melt(id_vars="index",var_name="actual",value_name="count").rename(columns={"index":"pred"})
sparse_conf_df["percent"] = sparse_conf_df["count"]/sparse_conf_df["count"].sum()
sparse_conf_df = sparse_conf_df.sort_values(by="count",ascending=False);sparse_conf_df

,pred,actual,count,percent
135,X,NOUN,10157.0,0.239620
271,X,VERB,5511.0,0.130013
33,X,ADP,4830.0,0.113947
101,X,DET,4576.0,0.107955
186,X,PRON,3740.0,0.088233
16,X,ADJ,3473.0,0.081934
67,X,AUX,2541.0,0.059946
50,X,ADV,1710.0,0.040342
169,X,PART,1261.0,0.029749
203,X,PROPN,997.0,0.023521


In [23]:
mistakes_df[(mistakes_df.predicted=="X")].head(100)

,predicted,actual,token,sentence,actual_tokens
0,X,PROPN,Creative,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
1,X,PROPN,Commons,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
2,X,NOUN,Corporation,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
3,X,AUX,is,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
4,X,PART,not,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
5,X,DET,a,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
6,X,NOUN,law,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
7,X,NOUN,firm,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
8,X,CCONJ,and,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT
9,X,AUX,does,Creative Commons Corporation is not a law firm and does not provide legal services .,PROPN PROPN NOUN AUX PART DET NOUN NOUN CCONJ AUX PART VERB ADJ NOUN PUNCT


For some reason the model predicts the X token in vast majority of cases, sometimes even for every word in a given example.